# AKSSF Anadromous Waters Catalog Data Join
Use spatial join to add AWC_CODE, Species and LSTAGE information to AKSSF catchments.  Add new field to concatenate species+lstage and field ot designate salmon spawning habitat presence or absence.

In [1]:
import os, arcpy, sys,datetime, traceback
import time

arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338)  #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr

print('imports complete')
print(f'{("-"*100)}')
print(f'sys paths {sys.path}')
print(f'{("-"*100)}')
print(f'Python Environment set to - {sys.base_exec_prefix}')
print(f'{("-"*100)}')
print (datetime.datetime.now())
outdir = os.path.dirname(os.getcwd())
print(f'CSV table output directory set to {outdir}')

imports complete
----------------------------------------------------------------------------------------------------
sys paths ['C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF\\data_preparation\\sensitivity_drivers\\geomorphology', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\Resources\\ArcPy', 'C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\python37.zip', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\DLLs', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\lib', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3', '', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\lib\\site-packages', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin', 'C:\\Users\\dwmerrigan\\A

## Functions

In [2]:
# Function to add key, value pairs to dictionary
def append_value(dict_obj, key, value):
    # Check if key exist in dict or not
    if key in dict_obj:
        # Key exist in dict.
        # Check if type of value of key is list or not
        if not isinstance(dict_obj[key], list):
            # If type is not list then make it list
            dict_obj[key] = [dict_obj[key]]
        # Append the value in list
        dict_obj[key].append(value)
    else:
        # As key is not in dict,
        # so, add key-value pair
        dict_obj[key] = value
# Function to remove parenthesis from user inputs
def replace_all(userinput, dic):
    for i, j in dic.items():
        userinput = userinput.replace(i, j)
    return userinput

# Getnull rows from numpy array
def getnull(cat_ID_con):
    nullRows = []
    nullRows.append(cat_ID_con)
    return True

#Generate unique column names
def uniquify(df_final):
    seen = set()
    for item in df_final:
        fudge = 1
        newitem = item
        while newitem in seen:
            fudge += 1
            newitem = "{}_{}".format(item, fudge)
        yield newitem
        seen.add(newitem)

## Section 1
### Set input datasets and output locations
User to input paths for necessary input data and output locations



In [66]:
# Get user inputs
# Used to format user inputs
inputDict = {"'":"",'"':""}# Specify path to AKSSF parent directory
while True:
    try:
        userinput = replace_all((input('Input path to temp site catchments fc.\nLeave blank and hit enter to use the default location.\nDefault = D:\\GIS\\AKSSF_land_met\\AKSSF_land_met.gdb\\all_cats_int_merge') or 'D:\\GIS\\AKSSF_land_met\\AKSSF_land_met.gdb\\all_cats_int_merge'),inputDict)
        if not arcpy.Exists(userinput):
            print('Path specified does not exist!\nPlease re-enter a valid path')
            continue
        else:
            catsint = userinput
            break
    except KeyboardInterrupt:
        print('interrupted!')
        sys.exit()
print(f'AKSSF temp site catchments {catsint}\n {"-"*100}')

# Specify path to AWC events fc
while True:
    try:
        userinput2 = replace_all((input('Input path to awc events feature class or shapefile.\nLeave blank and hit enter to use the default location.\nDefault = D:\\Basedata\\AWC\\AWC_2021_SpeciesEvents.gdb\\awcEventArcs') or "D:\\Basedata\\AWC\\AWC_2021_SpeciesEvents.gdb\\awcEventArcs"), inputDict)
        if not arcpy.Exists(userinput2):
            print('Path specified does not exist!\nPlease re-enter a valid path')
            continue
        else:
            awc_events = userinput2
            break
    except KeyboardInterrupt:
        print('interrupted!')
        sys.exit()
print(f'AWC events feature class set to {awc_events}\n {"-"*100}')

# Specify path to AWC Lakes events fc
while True:
    try:
        userinput4 = replace_all((input('Input path to awc lakes events feature class or shapefile.\nLeave blank and hit enter to use the default location.\nDefault = D:\\Basedata\\AWC\\AWC_2021_SpeciesEvents.gdb\\AWCLakeEvents') or "D:\\Basedata\\AWC\\AWC_2021_SpeciesEvents.gdb\\AWCLakeEvents"), inputDict)
        if not arcpy.Exists(userinput2):
            print('Path specified does not exist!\nPlease re-enter a valid path')
            continue
        else:
            awc_lakes = userinput4
            break
    except KeyboardInterrupt:
        print('interrupted!')
        sys.exit()
print(f'AWC lakes events feature class set to {awc_lakes}\n {"-"*100}')

# Specify path to output gdb
while True:
    try:
        userinput3 = replace_all((input('Input path to output gdb.\nLeave blank and hit enter to use the default location.\nDefault = C:\\Users\\dwmerrigan\\Documents\\GitHub\\ArcGIS_Default\\ArcGIS_Default.gdb') or "C:\\Users\\dwmerrigan\\Documents\\GitHub\\ArcGIS_Default\\ArcGIS_Default.gdb"), inputDict)
        if not arcpy.Exists(userinput2):
            print('Path specified does not exist!\nPlease re-enter a valid path')
            continue
        else:
            outputgdb = userinput3
            break
    except KeyboardInterrupt:
        print('interrupted!')
        sys.exit()
print(f'Output GDB set to {outputgdb}\n {"-"*100}')



AKSSF temp site catchments D:\GIS\AKSSF_land_met\AKSSF_land_met.gdb\all_cats_int_merge
 ----------------------------------------------------------------------------------------------------
AWC events feature class set to D:\Basedata\AWC\AWC_2021_SpeciesEvents.gdb\awcEventArcs
 ----------------------------------------------------------------------------------------------------
AWC lakes events feature class set to D:\Basedata\AWC\AWC_2021_SpeciesEvents.gdb\AWCLakeEvents
 ----------------------------------------------------------------------------------------------------
Output GDB set to C:\Users\dwmerrigan\Documents\GitHub\ArcGIS_Default\ArcGIS_Default.gdb
 ----------------------------------------------------------------------------------------------------


## Join awc data to catchments via spatial join
 * Add field 'sp_ls' to record species and l_stage comma seperated concatenations and field 'sa_sp_hab' to record presence absence of salmon spawning habitat

In [51]:
import time, datetime
arcpy.env.overwriteOutput = True
# Parameters
joinopt = "JOIN_ONE_TO_ONE"
jointype = "KEEP_ALL"
matchopt = "INTERSECT"
s_rad = "1 Centimeters"

processStart = time.time()

awcjoin = arcpy.analysis.SpatialJoin(catsint, awc_events, os.path.join(outputgdb,'cats_int_awc_spJoin'), joinopt, jointype, r'SourceFC "SourceFeatureClass" true true false 20 Text 0 0,First,#,catsint,SourceFC,0,20;AreaSqKm "AreaSqKm" true true false 8 Double 0 0,First,#,catsint,AreaSqKm,-1,-1;cat_ID_con "cat_ID_con" true true false 255 Text 0 0,First,#,catsint,cat_ID_con,0,255;MERGE_SRC "MERGE_SRC" true true false 255 Text 0 0,First,#,catsint,MERGE_SRC,0,255;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,catsint,Shape_Length,-1,-1,awc_events,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,catsint,Shape_Area,-1,-1;region "region" true true false 255 Text 0 0,First,#,catsint,region,0,255,awc_events,REGION,0,20;AWC_CODE "AWC_CODE" true false false 500 Text 0 0,Join,",",awc_events,AWC_CODE,0,70;SPECIES "SPECIES" true false false 500 Text 0 0,Join,",",awc_events,SPECIES,0,2;LSTAGE "LSTAGE" true false false 500 Text 0 0,Join,",",awc_events,LSTAGE,0,1', matchopt,s_rad, '')

arcpy.AddField_management(awcjoin,'sp_ls','TEXT')
arcpy.AddField_management(awcjoin,'sa_sp_hab','SHORT')
arcpy.AddField_management(awcjoin,'mult_awc','SHORT')
arcpy.AddField_management(awcjoin,'AWC_CODE_qc','TEXT',"","",500)

# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()
print(f'Process complete')

# Report success
print(f'Process completed at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} '
      f'(Elapsed time: {datetime.timedelta(seconds=processElapsed)})')
print(f'{"*"*100}')


Process complete
Process completed at 2022-02-16 13:57 (Elapsed time: 0:02:42)
****************************************************************************************************



## ~~Make dictionary from awc events~~
## Cannot use a dictionary because the awc records from spatial join do not match records in value Dictionary for all systems.



In [68]:
awcDict = {}
awcLakesDict = {}
with arcpy.da.SearchCursor(awc_events, ['AWC_CODE',"SPECIES","LSTAGE"]) as cur:
    for row in cur:
        sp_ls_val  = row[1] + "_" + row[2]
        append_value(awcDict,row[0],sp_ls_val)

with arcpy.da.SearchCursor(awc_lakes, ['AWC_CODE',"SPECIES","LSTAGE"]) as cur:
    for row in cur:
        sp_ls_val  = row[1] + "_" + row[2]
        append_value(awcLakesDict,row[0],sp_ls_val)
print(f'AWC Dictionaries Created\n{("*"*100)}')


AWC Dictionaries Created
****************************************************************************************************


## Use update cursor to populate new field


In [59]:
awcjoin = os.path.join(outputgdb,'cats_int_awc_spJoin')
dropfields = ['Join_Count', 'NHDPlusID','catID','Shape','TARGET_FID','SOURCEFC']
keepfields = [f.name for f in arcpy.ListFields(awcjoin) if f.name not in dropfields]
sa_sp_codes = ['CO_s','S_s','K_s', 'CH_s']
print(keepfields)
multirecslist = []
multirecsDict = {}
norecslist = []
sp_lsList = []
awcCodeDict = {}
catAwcDict = {}
curfields = ['cat_ID_con','AWC_CODE','Species', 'LSTAGE', 'sp_ls','sa_sp_hab','mult_awc','AWC_CODE_qc','SHAPE@']
c2 = 0
with arcpy.da.UpdateCursor(awcjoin, curfields) as cur:
    for row in cur:
        awcvals = []
        print(f'{(100*"*")}')
        if row[1] is not None:
            for val in row[1].split(','):
                val2 = "'"+str(val)+"'"
                awcvals.append(val)
            awcvals = list(set(awcvals))
            if len(str(awcvals).split(',')) > 1:
                print(f'Multiple awc records({len(str(awcvals).split(","))}) for this catchment ')
                append_value(multirecsDict,row[0],awcvals)
                row[6] = 1
                row[7] = str(awcvals).strip("[]").replace("'","")
            else:
                row[7]=awcvals[0]
                row[6] = 0
            spvals = row[2].split(",")
            lsvals = row[3].split(",")
            snum = len(row[2].split(","))
            cat_spList =[]
            for i in range(snum):
                sp_ls = spvals[i] + '_' + lsvals[i]
                append_value(awcCodeDict,row[1],sp_ls)
                append_value(catAwcDict,row[0],sp_ls)
                sp_lsList.append(sp_ls)
                cat_spList.append(sp_ls)
            print(f'{row[0]} has {snum} species_ls records {cat_spList}')
            print(f'list without duplicate records {list(set(cat_spList))}')
            row[4] = str(list(set(cat_spList))).strip("[]").replace("'","")
            if len(set(cat_spList).intersection(sa_sp_codes)) >0:
                print(f'{row[0]} has {set(cat_spList).intersection(sa_sp_codes)} salmon spawning records')
                row[5] = 1
            else:
                print(f'{row[0]} has no spawning habitat recorded')
                row[5] = 0
        else:
            print(f'{row[0]} did not intersect the AWC ')
            row[5] = 0
            row[6] = 0
            norecslist.append(row[0])
            c2+=1
        cur.updateRow(row)
    del(row)
del(cur)

print(f'{c2} catchments did not have AWC species records')

['OBJECTID', 'SourceFC', 'AreaSqKm', 'cat_ID_con', 'MERGE_SRC', 'region', 'AWC_CODE', 'SPECIES', 'LSTAGE', 'Shape_Length', 'Shape_Area', 'sp_ls', 'sa_sp_hab', 'mult_awc', 'AWC_CODE_qc']
****************************************************************************************************
Cook_Inlet_75004300006312 has 5 species_ls records ['DV_p', 'K_s', 'SH_p', 'CO_s', 'DV_r']
list without duplicate recrds ['DV_p', 'SH_p', 'DV_r', 'K_s', 'CO_s']
Cook_Inlet_75004300006312 has {'CO_s', 'K_s'} salmon spawning records
****************************************************************************************************
Cook_Inlet_75004300001906 has 3 species_ls records ['CO_s', 'DV_p', 'S_s']
list without duplicate recrds ['DV_p', 'S_s', 'CO_s']
Cook_Inlet_75004300001906 has {'S_s', 'CO_s'} salmon spawning records
****************************************************************************************************
Multiple awc records(3) for this catchment 
Cook_Inlet_75004300000100 has 13 spec

## Update using qc dictionaries

In [81]:
# cat_ID_con to corresponding AWC code from awcEventsArcs (2021) fc
awcQcDict = {'Bristol_Bay_2088326': '324-10-10150-2053',
             'Bristol_Bay_4054200': '325-30-10100-2202-3590',
             'Bristol_Bay_4059016': '325-30-10100-2031-3077-4001',
             'Bristol_Bay_4073375': '325-30-10100-2031-3118-4112',
             'Bristol_Bay_4076505': '325-30-10100-2031-3118-4006',
             'Bristol_Bay_4088904': '325-30-10100-2202-3080-4095',
             'Bristol_Bay_4089074': '325-30-10100-2202-3080-4086',
             'Bristol_Bay_4091164': '325-30-10100-2031-3118-4212-5051',
             'Bristol_Bay_4091424': '325-30-10100-2202-3080-4199',
             'Bristol_Bay_4092244': '325-30-10100-2202-3080-4210-5005',
             'Bristol_Bay_4098704': '325-30-10100-2202-3080',
             'Bristol_Bay_4101123': '325-30-10100-2249',
             'Kodiak_100826': '255-10-10010-2059',
             'Kodiak_103196': '256-20-10010-2016',
             'Kodiak_128685': '253-33-10010',
             'Kodiak_49617': '257-40-10030',
             'Kodiak_77794': '259-21-10120',
             'Kodiak_97276': '256-20-10010-2016-3007',
             'Prince_William_Sound_28086': '226-30-16810',
             'Prince_William_Sound_29854': '221-10-10020-2003',
             'Prince_William_Sound_31865': '227-20-17480',
             'Prince_William_Sound_38993': '221-30-10620',
             'Prince_William_Sound_43383': '221-30-10516',
             'Prince_William_Sound_43933': '221-20-10360',
             'Prince_William_Sound_43973': '221-20-10350',
             'Prince_William_Sound_46513': '221-10-10160',
             'Cook_Inlet_75004300000100': '244-10-10010-2021',
             'Cook_Inlet_75004300004304': '244-10-10010',
             'Cook_Inlet_75004300002332': '244-10-10010',
             'Cook_Inlet_75004300005303': '244-10-10010-2011-3031',
             'Cook_Inlet_75004300001207': '244-10-10010-2025',
             'Cook_Inlet_75004300004192': '244-10-10010',
             'Cook_Inlet_75004300007324': '244-10-10010-2011-3031',
             'Cook_Inlet_75004300001162': '244-10-10010-2011',
             'Cook_Inlet_75004300003171': '244-10-10010-2011-3031',
             'Cook_Inlet_75004300004341': '244-10-10010',
             'Cook_Inlet_75004300002289': '244-10-10010-2088',
             'Cook_Inlet_75004300000105': '244-10-10050',
             'Cook_Inlet_75004300006013': '244-30-10050-2059',
             'Cook_Inlet_75004400000576': '244-30-10010-2177',
             'Cook_Inlet_75004400010402': '244-30-10010-2025-3041',
             'Cook_Inlet_75004400010754': '244-30-10010-2158',
             'Cook_Inlet_75004400007385': '244-30-10010-2039',
             'Cook_Inlet_75004400007948': '244-30-10010-2177',
             'Cook_Inlet_75004400010320': '244-30-10010-2030',
             'Cook_Inlet_75004400001568': '244-30-10010-2050',
             'Cook_Inlet_75004400008917': '244-30-10010-2025',
             'Cook_Inlet_75004400001952': '244-30-10010',
             'Cook_Inlet_75004400007388': '244-30-10010-2025',
             'Cook_Inlet_75004400003042': '247-90-10020',
             'Cook_Inlet_75004400002041': '244-30-10010-2177-3012',
             'Cook_Inlet_75004400011322': '247-60-10170-2029-3033',
             'Cook_Inlet_75000100000981': '247-50-10260-2019',
             'Cook_Inlet_75000100002463': '247-50-10060',
             'Cook_Inlet_75000100004336': '247-50-10050',
             'Cook_Inlet_75000100004200': '247-50-10260-2019',
             'Cook_Inlet_75000100000059': '247-60-10320',
             'Cook_Inlet_75000400000173': '247-50-10200-2081',
             'Cook_Inlet_75000400014405': '247-50-10200-2071',
             'Cook_Inlet_75000400014403': '247-50-10220-2085',
             'Cook_Inlet_75000500009621': '247-41-10200-2381-3260',
             'Cook_Inlet_75000600010189': '247-41-10200-2053',
             'Cook_Inlet_75000200013536': '247-41-10200',
             'Cook_Inlet_75000200005504': '247-41-10200-2081',
             'Cook_Inlet_75000200002930': '247-41-10200-2081-3100',
             'Cook_Inlet_75000200010144': '247-41-10200-2081-3065',
             'Cook_Inlet_75000200002753': '247-41-10200-2081',
             'Cook_Inlet_75000200015584': '247-41-10200-2081-3100-4167',
             'Cook_Inlet_75000200007839': '247-41-10200-2081',
             'Cook_Inlet_75000200007958': '247-41-10200-2081-3100-4167-5044',
             'Cook_Inlet_75000200013189': '247-41-10200-2081-3100',
             'Cook_Inlet_75000200000213': '247-41-10200-2081',
             'Cook_Inlet_75000200015782': '247-41-10200-2081-3073',
             'Cook_Inlet_75000200007852': '247-41-10200-2081',
             'Cook_Inlet_75000200007660': '247-41-10100',
             'Cook_Inlet_75000200008734': '247-41-10100-2231-3050',
             'Cook_Inlet_75000200015587': '247-41-10200-2081-3100-4167',
             'Cook_Inlet_75000200015320': '247-41-10100',
             'Cook_Inlet_75000200012046': '247-41-10100-2343',
             'Cook_Inlet_75000200005444': '247-41-10200-2081-3130',
             'Cook_Inlet_75000200018203': '247-41-10200-2081-3100-4155-5014',
             'Cook_Inlet_75000200013249': '247-41-10200-2081-3100',
             'Cook_Inlet_75000200008508': '247-41-10100',
             'Cook_Inlet_75000200007670': '247-41-10100-2329',
             'Cook_Inlet_75000200018667': '247-41-10100',
             'Cook_Inlet_75000200011034': '247-41-10100-2150',
             'Copper_River_75019800010313': '212-20-10080-2655',
             'Copper_River_75019700004190': '212-20-10080-2300-3421',
             'Copper_River_75019700003889': '212-20-10080-2300-3421-4062',
             'Copper_River_75003900028507': '212-10-10030-2050',
             'Copper_River_75003900027489': '212-10-10010-2061-3025',
             'Copper_River_75003900044738': '212-10-10010-2041-3010',
             'Copper_River_75003900055316': None,
             'Copper_River_75003900039073': '228-70-10500',
             'Copper_River_75003900027771': '212-10-10010-2052'
             }
# cat_ID_con to corresponding AWC code from AWCLakeEvents (2021) fc
awcLakesQcLakesDict = {'Bristol_Bay_2087913': '324-10-10150-2207-0030',
                       'Bristol_Bay_2068072': '324-10-10150-2207-0030'
                       }



Try Select layer by attribute (AWC_CODE) for associated catcment then sub_selection by location using catchment shape.  Then use search cursor to build sp_ls list.



In [93]:
sa_sp_codes = ['CO_s','S_s','K_s', 'CH_s']
fields = ['cat_ID_con','AWC_CODE_qc','sp_ls','sa_sp_hab']
c=1
with arcpy.da.UpdateCursor(awcjoin,fields) as cur:
    for row in cur:
        if row[0] in awcQcDict:
            print(f'{("*"*100)}\n')
            print(f'{c}. {row[0]} found in awc qc dictionary')
            updCode = awcQcDict[row[0]]
            if updCode is not None:
                updSpLS = str(awcDict[updCode]).strip("[]").replace("'","")
                print (f'Awc codes {row[1]} will be updated to {updCode}')
                print (f'Sp_ls codes {row[2]} will be updated to {updSpLS}')
                if len(set(awcDict[updCode]).intersection(sa_sp_codes)) >0:
                    print(f'{row[0]} has {len(set(awcDict[updCode]).intersection(sa_sp_codes))} salmon spawning records')
                    row[3] = 1
                else:
                    print(f'{row[0]} has no spawning habitat recorded')
                    row[3] = 0
                row[1] = updCode
                row[2] = updSpLS
            else:
                print(f'{("*"*100)}\n')
                print(f'{row[0]} will have code removed {updCode}')
                row[1]=None
                row[2]=None
                row[3]=None
            c+=1
        elif row[0] in awcLakesQcLakesDict:
            print(f'{("*"*100)}\n')
            print(f'{c}. {row[0]} found in awc lakes qc dictionary')
            print(f'{row[0]} found in awc qc dictionary')
            updCode = awcLakesQcLakesDict[row[0]]
            updSpLS = str(awcLakesDict[updCode]).strip("[]").replace("'","")
            print (f'Awc codes {row[1]} will be updated to {updCode}')
            print (f'Sp_ls codes {row[2]} will be updated to {updSpLS}')
            if len(set(awcLakesDict[updCode]).intersection(sa_sp_codes)) >0:
                print(f'{row[0]} has {len(set(awcLakesDict[updCode]).intersection(sa_sp_codes))} salmon spawning records')
                row[3] = 1
            else:
                print(f'{row[0]} has no spawning habitat recorded')
                row[3] = 0
            row[1] = updCode
            row[2] = updSpLS
            c+=1
        else:
            pass
        cur.updateRow(row)
    del(row)
del(cur)

****************************************************************************************************

1. Cook_Inlet_75004300000100 found in awc qc dictionary
Awc codes 244-10-10010, 244-10-10010-2021, 244-10-10010-2021-3009 will be updated to 244-10-10010-2021
Sp_ls codes DV_p, P_p, SH_p, DV_r, CO_r, K_s, CO_s, K_r, SH_r will be updated to CO_s, DV_r, CO_r, K_s, SH_r, K_r
Cook_Inlet_75004300000100 has 2 salmon spawning records
****************************************************************************************************

2. Cook_Inlet_75004300004304 found in awc qc dictionary
Awc codes 244-10-10010, 244-10-10010-2090 will be updated to 244-10-10010
Sp_ls codes DV_p, SH_p, DV_r, CO_p, CO_r, K_s, CO_s, K_r will be updated to DV_p, SH_p, K_s, CO_r, P_s, CO_r, S_p, SH_s, CO_p, CH_p, CO_s, K_r, P_p
Cook_Inlet_75004300004304 has 2 salmon spawning records
****************************************************************************************************

3. Cook_Inlet_75004300002332 fo

In [97]:
print(f'{awcDict["244-10-10010"]}\n{len(awcDict["244-10-10010"])}')

['DV_p', 'SH_p', 'K_s', 'CO_r', 'P_s', 'CO_r', 'S_p', 'SH_s', 'CO_p', 'CH_p', 'CO_s', 'K_r', 'P_p']
13
